In [4]:
from bs4 import BeautifulSoup
import requests

# similar to our API requests, but now we just want all the text
html = requests.get('https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_2021').text
soup = BeautifulSoup(html, 'html.parser')
print(soup.prettify())
table = soup.find('table', {"class":'wikitable sortable'})

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Billboard Year-End Hot 100 singles of 2021 - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"82cfde49-55e7-44ab-af64-c777f84e0d39","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Billboard_Year-End_Hot_100_singles_of_2021","wgTitle":"Billboard Year-End Hot 100 singles of 2021","wgCurRevisionId":1083457993,"wgRevisionId":1083457993,"wgArticleId":69418946,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["2021 record charts","Billboard charts"],"wgPage

Wikipedia pages

In [5]:
import pandas as pd

years = [2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012]

billboard = pd.DataFrame()
for year in years:
    url = 'https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_' + str(year)
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')
    #print(soup.prettify())
    table = soup.find('table', {"class":'wikitable sortable'})
    df = pd.read_html(str(table))
    df = pd.concat(df)
    df['Year'] = str(year)
    billboard = pd.concat([billboard, df], ignore_index=True)
    
billboard
billboard.to_csv("billboard.csv")

Spotify

In [63]:
# Spotify Headers
CLIENT_ID = "914d599079cf414bb7441d9c09e8dd1d"
CLIENT_SECRET = "c67d0eee9ade4d49a710acf1a7615a3d"

AUTH_URL = "https://accounts.spotify.com/api/token"
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

#Convert response to JSON
auth_response_data = auth_response.json()

#Save the access token
access_token = auth_response_data['access_token']

#Need to pass access token into header to send properly formed GET request to API server
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

BASE_URL = 'https://api.spotify.com/v1/'

In [62]:
name = []
spo_id = []
explicit = []
images = []
release_date = []
preview_url = []

In [77]:
for index in billboard.index:
    if index < 895: continue
    print(index)
    title = billboard["Title"][index]
    artists = billboard["Artist(s)"][index]
    link = BASE_URL + 'search?q=' + title + '&type=track'
    r = requests.get(link, headers=headers)
    r = r.json()
    name.append(r["tracks"]['items'][0]["name"])
    spo_id.append(r["tracks"]['items'][0]["id"])
    explicit.append(r["tracks"]['items'][0]["explicit"])
    images.append(r["tracks"]['items'][0]['album']["images"][0]['url'])
    release_date.append(r["tracks"]['items'][0]['album']["release_date"])
    preview_url.append(r["tracks"]['items'][0]["preview_url"])

895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
980
981
982
983
984
985
986
987
988
989
990
991
992
993
994
995
996
997
998
999


In [74]:
billboard['spo_name'] = name
billboard['spo_id'] = spo_id
billboard['explicit'] = explicit
billboard['images'] = images
billboard['release_date'] = release_date
billboard['preview_url'] = preview_url

In [76]:
billboard.to_csv("bill_spo.csv")

Apple Music

In [41]:
name_a = []
apple_id = []
genre = []
trackTime = []
trackView = []

In [6]:
import re
import time

In [56]:
for index in billboard.index:   
    title = billboard["Title"][index].replace('"', '')
    artists = billboard["Artist(s)"][index]
    link = "https://itunes.apple.com/search?term=" + re.sub('[^a-zA-Z0-9 \n\.]', '', title) + " " + artists + "&media=music"
    r = requests.get(link)
    r = r.json()
    name_a.append(r["results"][0]["trackName"])
    apple_id.append(r["results"][0]["trackId"])
    genre.append(r["results"][0]['primaryGenreName'])
    trackTime.append(r["results"][0]["trackTimeMillis"])
    trackView.append(r["results"][0]["trackViewUrl"])
    time.sleep(10)

971
<Response [200]>
972
<Response [200]>
973
<Response [200]>
974
<Response [200]>
975
<Response [200]>
976
<Response [200]>
977
<Response [200]>
978
<Response [200]>
979
<Response [200]>
980
<Response [200]>
981
<Response [200]>
982
<Response [200]>
983
<Response [200]>
984
<Response [200]>
985
<Response [200]>
986
<Response [200]>
987
<Response [200]>
988
<Response [200]>
989
<Response [200]>
990
<Response [200]>
991
<Response [200]>
992
<Response [200]>
993
<Response [200]>
994
<Response [200]>
995
<Response [200]>
996
<Response [200]>
997
<Response [200]>
998
<Response [200]>
999
<Response [200]>


In [573]:
billboard['itunes_name'] = name_a
billboard['apple_id'] = apple_id
billboard['genre'] = genre
billboard['trackTime'] = trackTime
billboard['trackView'] = trackView

In [574]:
billboard.to_csv("bill_spo_itunes.csv", orient="records")
billboard.to_csv("billboard.json", )

In [99]:
billboard

,No.,Title,Artist(s),Year,spo_name,spo_id,explicit,images,release_date,preview_url,itunes_name,apple_id,genre,trackTime,trackView
0,1,"""Levitating""",Dua Lipa,2021,Levitating,5nujrmhLynf4yMoMtj8AQF,False,https://i.scdn.co/image/ab67616d0000b273bd26ed...,2020-03-27,https://p.scdn.co/mp3-preview/130e18a8f12e319b...,Levitating (feat. DaBaby),1532788352,Pop,203064,https://music.apple.com/us/album/levitating-fe...
1,2,"""Save Your Tears""",The Weeknd and Ariana Grande,2021,Save Your Tears (with Ariana Grande) (Remix),5QO79kh1waicV47BqGRL3g,True,https://i.scdn.co/image/ab67616d0000b2738863bc...,2020-03-20,NaN,Save Your Tears,1563812775,R&B/Soul,191014,https://music.apple.com/us/album/save-your-tea...
2,3,"""Blinding Lights""",The Weeknd,2021,Blinding Lights,0VjIjW4GlUZAMYd2vXMi3b,False,https://i.scdn.co/image/ab67616d0000b2738863bc...,2020-03-20,NaN,Blinding Lights,1499378607,R&B/Soul,200046,https://music.apple.com/us/album/blinding-ligh...
3,4,"""Mood""",24kGoldn featuring Iann Dior,2021,Mood (feat. iann dior),4jPy3l0RUwlUI9T5XHBW2m,True,https://i.scdn.co/image/ab67616d0000b27384c53f...,2021-03-26,https://p.scdn.co/mp3-preview/9f4d2e78c5f0ebb3...,Mood (feat. iann dior),1584016415,Hip-Hop/Rap,140885,https://music.apple.com/us/album/mood-feat-ian...
4,5,"""Good 4 U""",Olivia Rodrigo,2021,good 4 u,4ZtFanR9U6ndgddUvNcjcG,True,https://i.scdn.co/image/ab67616d0000b273a91c10...,2021-05-21,NaN,good 4 u,1560735551,Pop,178148,https://music.apple.com/us/album/good-4-u/1560...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,96,"""Somethin' 'Bout a Truck""",Kip Moore,2012,Somethin' 'Bout A Truck,6Ymvlzom4TQeoKqAWsZRD8,False,https://i.scdn.co/image/ab67616d0000b27384550c...,2012-01-01,NaN,Somethin' 'Bout a Truck,1440809921,Country,215147,https://music.apple.com/us/album/somethin-bout...
996,97,"""Adorn""",Miguel,2012,Adorn,25cUhiAod71TIQSNicOaW3,False,https://i.scdn.co/image/ab67616d0000b273fe113f...,2012-09-28,https://p.scdn.co/mp3-preview/91de8c966c9a2df2...,Adorn (feat. Wiz Khalifa),561709114,R&B/Soul,227067,https://music.apple.com/us/album/adorn-feat-wi...
997,98,"""Fly Over States""",Jason Aldean,2012,Fly Over States,17X2tSzibD9n9QRjdExcWT,False,https://i.scdn.co/image/ab67616d0000b27330fb80...,2010-11-02,https://p.scdn.co/mp3-preview/bab5c939f0f7205f...,Fly Over States,1263704273,Country,218200,https://music.apple.com/us/album/fly-over-stat...
998,99,"""Even If It Breaks Your Heart""",Eli Young Band,2012,Even If It Breaks Your Heart,0Nu9WA8kEbBWEsay2s8Q0U,False,https://i.scdn.co/image/ab67616d0000b273bf969d...,2011-01-01,NaN,Even If It Breaks Your Heart,1440794228,Country,220520,https://music.apple.com/us/album/even-if-it-br...
